In [1]:
import sys
import json
import requests
import re
import numpy as np
import pandas as pd
import pymysql as pm
# 쿼리문 그대로 박아서 쓸 수 있음 데이터 읽어들일 때 사용 
from sqlalchemy import create_engine
# 데이터프레임 단위로 mysql db에 적재 가능, 만든거 db에 박을때 사용 가능 
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
import re
import math as m
from dateutil.relativedelta import relativedelta
from google.cloud import bigquery

/Users/gowid/Library/Python/3.8/lib/python/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [2]:
client_gowid_gcs_dev = bigquery.Client(project='gowid-gcs-dev')

In [3]:

if __name__ == '__main__':
    sql_path = './sql/'
    
    def sql_card(_file_name):
        with open(str(sql_path)+_file_name, 'r') as q:
            sql = q.read()
            return client_gowid_gcs_dev.query(sql)

In [4]:
# pg_tracker base table 가져오기
base = sql_card('pg_tracker.sql').to_dataframe()

In [5]:
#dictionary 가공
# 불러오고 include / exclude 나누기
pg_dic = sql_card('dic_pg_tracker.sql').to_dataframe()
pg_dic_in = pg_dic.loc[pg_dic['pg_category'] == 'Include']
pg_dic_ex = pg_dic.loc[pg_dic['pg_category'] == 'Exclude']
# include -> str 처리
pg_dic_in_str = pg_dic_in[['pg_desc']].to_string(header=False, index = False, index_names = False).split('\n')
pg_dic_in_str = ",".join(pg_dic_in_str).replace(",","|").replace(" ","")
# exclude -> str 파라미터 처리
pg_dic_ex_str = pg_dic_ex[['pg_desc']].to_string(header=False, index = False, index_names = False).split('\n')
pg_dic_ex_str = ",".join(pg_dic_ex_str).replace(",","|").replace(" ","")

In [6]:
# pg 매출과 일반매출 분류
base['pg_flag'] = (base['resaccount_final'].str.contains(pg_dic_in_str) == True) & (base['resaccount_final'].str.contains(pg_dic_ex_str) == False)
base.groupby(['pg_flag'])['pg_flag'].count()

pg_flag
False    4070082
True      415808
Name: pg_flag, dtype: int64